# HackUMBC 2022: DeFi Credibility 

Users of cryptocurrency are constantly targeted by various fraud and various threats on their holdings. Because of a lack of regulation and knowledge in the crypto space, attackers have been able to exploit protocols and users alike, resulting in over $10 billion in theft and scams in 2021 alone. Of these attacks, roughly $38 million was stolen from Alpha Homora V2, a yield farming protocol. One method of helping to protect companies and consumers, along with their assets is to instill trust in the parties with whom they are transacting with.

This notebook analyzes the aforementioned attack, providing a brief analysis on the transactions involved. Our goal is to help mitigate scams by creating a tool to determine the trustworthiness of cryptocurrency addresses.  

## Install Dependencies

In [1]:
!cd hackUMBC2022 && make install

pip install --upgrade pip &&\
pip install -r requirements.txt


## Imports

In [2]:
import pandas as pd
import scipy
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import os
import requests
from dotenv import load_dotenv
import json
from bs4 import BeautifulSoup as BS
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem


## API: Gathering ETH Transactions

Through the use of both the Covalent and Etherscan APIs, we are able to web scrape transaction data, then save it as a JSON file to later process and visualize the Alpha Homora exploit.

Attack Post-Mortem: https://medium.com/cream-finance/alpha-homora-v2-exploit-post-mortem-344d277bdea6

In [3]:
load_dotenv()
api_key = os.getenv("COVALENT_KEY")

def get_transactions(address) -> requests.Response:
    """Get transactions of a given ETH address"""
    BASE_URL = "https://api.covalenthq.com"
    transactions_endpoint = (
        BASE_URL + f"/v1/1/address/{address}/transactions_v2/?key={api_key}"
    )
    params = {
        "quote-currency": "USD",
        "no-logs": True
    }
    transactions = requests.get(transactions_endpoint, params=params, timeout=15)
    return transactions

In [4]:
address ="0x905315602ed9a854e325f692ff82f58799beab57"
transaction = get_transactions(address)
with open("transaction.json", "w", encoding="utf-8") as outfile:
    json.dump(transaction.json(), outfile, indent = 4, sort_keys=True)

## Data Cleaning

We parse the JSON file to convert it to CSV format, as well as remove some unnecessary data. 

In [5]:
with open("transaction.json", "r") as f:
    transactions = json.load(f)
transaction_array = transactions['data']['items']
transaction_df = pd.DataFrame(transaction_array)
transaction_df = transaction_df.drop(['from_address_label', 'to_address_label', 'fees_paid', 'gas_offered', 'gas_price', 'gas_quote', 'gas_quote_rate', 'gas_spent'], axis=1)
transaction_df.to_csv('transaction.csv')

## Data Visualization

### Count of Records to Different Wallet Addresses

This bar chart's purpose is to display the number of wallet addresses that received cryptocurrency in the given transactions. This information is significant because it shows the exploiter's address and the exchange they used as having the most transactions, giving us a visual representation of where they are sending the funds they stole. In this instance, the exploiter had a lot of transactions sent to Tornado.Cash, which is a mixer commonly used to launder funds. Through this, we are able to consider risk with this wallet address.

Attacker address: 0x905315602ed9a854e325f692ff82f58799beab57

Exchange address: 0xa160cdab225685da1d56aa342ad8841c3b53f291

In [6]:
_deepnote_run_altair(transaction_df, """{"data":{"name":"placeholder"},"mark":{"type":"bar","tooltip":true},"height":220,"$schema":"https://vega.github.io/schema/vega-lite/v4.json","autosize":{"type":"fit"},"encoding":{"x":{"sort":null,"type":"nominal","field":"to_address","scale":{"type":"linear","zero":false}},"y":{"sort":null,"type":"quantitative","field":"COUNT(*)","scale":{"type":"linear","zero":true},"aggregate":"count"},"color":{"sort":null,"type":"nominal","field":"","scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

### Count of Records from Different Wallet Addresses

The purpose of this bar chart is to display the addresses that the exploiter interacted with, which includes their address and, in this case, other users' messages to the hacker.

In [7]:
_deepnote_run_altair(transaction_df, """{"data":{"name":"placeholder"},"mark":{"type":"bar","tooltip":true},"height":220,"$schema":"https://vega.github.io/schema/vega-lite/v4.json","autosize":{"type":"fit"},"encoding":{"x":{"sort":null,"type":"nominal","field":"from_address","scale":{"type":"linear","zero":false}},"y":{"sort":null,"type":"quantitative","field":"COUNT(*)","scale":{"type":"linear","zero":true},"aggregate":"count"},"color":{"sort":null,"type":"nominal","field":"","scale":{"type":"linear","zero":false}}}}""")

alt.Chart(...)

### Displaying Trustworthiness

This is an early version of what we came up with in order to build confidence. Confidence amongst consumers for their personal transactions and confidence amongst companies looking to mitigate transactional risk.

In [8]:
def get_labels(address: str):
    """Get labels deeming untrusworthiness (if any)"""
    url = f"https://etherscan.io/address/{address}"

    software_names = [SoftwareName.CHROME.value]
    operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]
    user_agent_rotator = UserAgent(
        software_names=software_names, operating_systems=operating_systems, limit=100
    )
    user_agent = user_agent_rotator.get_random_user_agent()
    headers = {"User-Agent": user_agent}
    res = requests.get(url, headers=headers, timeout=15)
    if res.status_code != 200:
        print(f"Connection unsuccessful (Error code {res.status_code})")
        return None
    soup = BS(res.text, "html.parser")
    a_tags = soup.find_all("a", href=True)
    with open("user.txt", "w", encoding="utf-8") as f:
        f.write(str(a_tags))
    labels = []
    for a_tag in a_tags:
        if "/accounts/label" in a_tag["href"]:
            label = a_tag["href"].split("/")[-1]
            labels.append(label)
    return labels

The images below provide the output label that define our exploiter's wallet address, as well as another wallet that was involved in an attack being flagged with a 'heist' label.

![get_labels() output](/hackUMBC2022/images/get_labels_output_ex.png)
![more get_labels() output](/hackUMBC2022/images/get_labels_output_ex2.png)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4beb2dee-7648-4358-9d54-16785e02bd6c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>